## Symengineの検証

In [18]:
import symengine as sm
import sympy as smp
import numpy as np
import itertools
import math
from functools import cmp_to_key

In [2]:
def g(x):
    return sm.tanh(x)

def dg(x):
    return 1 - sm.tanh(x) * sm.tanh(x)

def dg_np(x):
    return 1 - np.tanh(x) * np.tanh(x)

def ddg(x):
    return 2 * (sm.tanh(x) * sm.tanh(x) - 1) * sm.tanh(x)

In [3]:
# https://stackoverflow.com/questions/22857162/multivariate-taylor-approximation-in-sympy

def mv_series(function_expression, 
              variable_list, 
              evaluation_point, 
              degree):
    """ Multivariate version of sympy series """
    
    n_var = len(variable_list)
    
    # dict of variables and their evaluation_point coordinates, ready for the subs() method
    point_coordinates = {i : j for i, j in zip(variable_list, evaluation_point) }
    
    # list with exponents of the partial derivatives
    deriv_orders = list(itertools.product(range(degree + 1), repeat=n_var))
    
    # Discarding some higher-order terms
    deriv_orders = [deriv_orders[i] for i in range(len(deriv_orders)) if sum(deriv_orders[i]) <= degree]
    n_terms = len(deriv_orders)
    
    # same as before, but now ready for the diff() method
    deriv_orders_as_input = [list(sum(list(zip(variable_list, deriv_orders[i])), ())) for i in range(n_terms)]
    
    polynomial = 0
    for i in range(n_terms):
        partial_derivatives_at_point = function_expression.diff(*deriv_orders_as_input[i]).subs(point_coordinates)
        # e.g. df/(dx*dy**2) evaluated at (x0,y0)
        
        denominator = math.prod([math.factorial(j) for j in deriv_orders[i]])
        # e.g. (1! * 2!)
        
        distances_powered = math.prod([(sm.Matrix(variable_list) - sm.Matrix(evaluation_point))[j] ** deriv_orders[i][j] for j in range(n_var)])  
        # e.g. (x-x0)*(y-y0)**2
        
        polynomial += partial_derivatives_at_point / denominator * distances_powered
    return polynomial


In [4]:
x_dim = 3
r_dim = 4
#r_dim = 100

In [5]:
# Symbols

x_list = [sm.Symbol(f'x_{i}') for i in range(x_dim)]
x_dot_list = [sm.Symbol('\dot{x}_' + f'{i}') for i in range(x_dim)]

x = sm.Matrix([[x_i] for x_i in x_list])
x_dot = sm.Matrix([[x_dot_i] for x_dot_i in x_dot_list])

In [6]:
# Concrete values
def calc_scale_with_spectral_radius(mat, rho):
    cur_spectral_radius = max(abs(np.linalg.eigvals(mat)))
    return rho / cur_spectral_radius

rng = np.random.RandomState(0)

r_s_np = rng.uniform(low=-0.5, high=0.5, size=(r_dim, 1))

x_s_np = np.zeros((x_dim, 1))

B_scale = 0.1
B_np = (rng.uniform(low=-1.0, high=1.0, size=(r_dim, x_dim)) * B_scale)

A_np = rng.uniform(low=-1.0, high=1.0, size=(r_dim, r_dim))
sparsify = np.vectorize(lambda v: v if rng.random() < 0.05 else 0.0)
A_np = sparsify(A_np)

rho = 0.01
A_scale = calc_scale_with_spectral_radius(A_np, rho)
print(f'A_scale={A_scale}')
#A_np = A_np * A_scale

d_np = np.zeros((r_dim, 1))

d_s_np = A_np @ r_s_np + d_np

ArBxd_np = A_np @ r_s_np + B_np @ x_s_np + d_s_np

# numpyはshapeがあっていなくてもアダマール積が可能なので利用
A_s_np = dg_np(ArBxd_np) * A_np - np.eye(r_dim)

A_s_inv_np = np.linalg.inv(A_s_np)

A_scale=inf


/var/folders/g8/lw05g79s3yv331w4vf305fkh0000gn/T/ipykernel_60961/506138365.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  return rho / cur_spectral_radius


In [7]:
def to_sm_matrix(np_mat):
    return sm.Matrix(np_mat.tolist())

B       = to_sm_matrix(B_np)
d_s     = to_sm_matrix(d_s_np)
r_s     = to_sm_matrix(r_s_np)
A_s_inv = to_sm_matrix(A_s_inv_np)
A       = to_sm_matrix(A_np)
gamma   = 100

In [8]:
Bx_d_s = B * x + d_s

expr0 = Bx_d_s.applyfunc(dg).multiply_elementwise(A * r_s)
expr1 = Bx_d_s.applyfunc(g)
expr2 = Bx_d_s.applyfunc(ddg).multiply_elementwise(A * r_s)
expr3 = Bx_d_s.applyfunc(dg)

expr2_3 = (expr2 - expr3).multiply_elementwise(B * x_dot)

expr = A_s_inv * (expr0 - expr1) + (1 / gamma) * (A_s_inv * A_s_inv) * expr2_3

x_zero = [0] * x_dim
x_dot_zero = [0] * x_dim

degree = 2

In [9]:
def get_poly_dict(p, var_list):
    """ 
    When making poly symengine expression, error occurs with var list arguemnt poly(expr, var_list),
    so we made workaround to reorder coeffs.
    """
    def reoder_key(key, indices):
        ret = [-1] * len(var_list)
        for i,k in enumerate(key):
            ret[indices[i]] = k
        return tuple(ret)
    
    poly_vars = p.args[1:]
    indices = []
    for v in var_list:
        index = poly_vars.index(v)
        indices.append(index)
    d = p.as_dict()
    new_d = {}
    for key,value in d.items():
        new_key = reoder_key(key, indices)
        new_d[new_key] = value
    return new_d

In [19]:
def compare(a, b):
    if a == b: 
        return 0
    if a > b:
        return 1
    else:
        return -1

def compare_degrees(degrees0, degrees1):
    def find_first_hit_pos(degrees):
        for i,degree in enumerate(degrees):
            if degree != 0:
                return i
        return -1
    
    var_size = len(degrees0) // 2

    xs0 = degrees0[:var_size]
    xs1 = degrees1[:var_size]    
    xdots0 = degrees0[var_size:]    
    xdots1 = degrees1[var_size:]
    
    x_max_degree0 = max(xs0)
    x_max_degree1 = max(xs1)
    xdot_max_degree0 = max(xdots0)
    xdot_max_degree1 = max(xdots1)
    
    x_sum_degree0 = sum(xs0)
    x_sum_degree1 = sum(xs1)    
    xdot_sum_degree0 = sum(xdots0)
    xdot_sum_degree1 = sum(xdots1)
    
    x_first_hit0 = find_first_hit_pos(xs0)
    x_first_hit1 = find_first_hit_pos(xs1)
    xdot_first_hit0 = find_first_hit_pos(xdots0)
    xdot_first_hit1 = find_first_hit_pos(xdots1)
    
    ret = compare(xdot_sum_degree0, xdot_sum_degree1)
    if ret != 0:
        return ret
    
    ret = compare(x_sum_degree0, x_sum_degree1)
    if ret != 0:
        return ret    
    
    ret = compare(x_first_hit0, x_first_hit1)
    if ret != 0:
        return ret
        
    ret = compare(x_max_degree0, x_max_degree1)
    if ret != 0:
        return -ret
        
    ret = compare(xdot_sum_degree0, xdot_sum_degree1)
    if ret != 0:
        return ret    
    
    ret = compare(xdot_first_hit0, xdot_first_hit1)
    if ret != 0:
        return ret    
        
    ret = compare(xdot_max_degree0, xdot_max_degree1)
    if ret != 0:
        return -ret
        
    return 0


In [22]:
%%time

coeff_dicts = []

for r_index in range(r_dim):
    r_ex = mv_series(expr[r_index,0], 
                     x_list + x_dot_list, 
                     x_zero + x_dot_zero, 
                     degree)
                 
    # ここはsympyを使う
    #p_c = smp.poly(r_ex, xs + x_dots)
    #coeff_dicts.append(p_c.as_dict())
    p_c = smp.poly(r_ex)
    coeff_dicts.append(get_poly_dict(p_c, x_list + x_dot_list))

CPU times: user 25.7 ms, sys: 2.2 ms, total: 28 ms
Wall time: 28.1 ms


In [23]:
degrees_sets = set()

for coeff_dict in coeff_dicts:
    for degrees in coeff_dict.keys():
        degrees_sets.add(degrees)

In [24]:
degrees_list = sorted(list(degrees_sets), key=cmp_to_key(compare_degrees))

degrees_list  

[(0, 0, 0, 0, 0, 0),
 (1, 0, 0, 0, 0, 0),
 (0, 1, 0, 0, 0, 0),
 (0, 0, 1, 0, 0, 0),
 (2, 0, 0, 0, 0, 0),
 (1, 1, 0, 0, 0, 0),
 (1, 0, 1, 0, 0, 0),
 (0, 2, 0, 0, 0, 0),
 (0, 1, 1, 0, 0, 0),
 (0, 0, 2, 0, 0, 0),
 (0, 0, 0, 1, 0, 0),
 (0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 1),
 (1, 0, 0, 1, 0, 0),
 (1, 0, 0, 0, 1, 0),
 (1, 0, 0, 0, 0, 1),
 (0, 1, 0, 1, 0, 0),
 (0, 1, 0, 0, 1, 0),
 (0, 1, 0, 0, 0, 1),
 (0, 0, 1, 1, 0, 0),
 (0, 0, 1, 0, 1, 0),
 (0, 0, 1, 0, 0, 1)]

In [46]:
all_coeffs = []

for coeff_dict in coeff_dicts:
    coeffs = []
    for degrees in degrees_list:

        if degrees in coeff_dict:
            coeffs.append(coeff_dict[degrees])
        else:
            coeffs.append(0)            
    all_coeffs.append(coeffs)
        
all_coeffs = np.array(all_coeffs, dtype=np.float64)

In [50]:
target_coeff_dicts = [{
    (0,1,0,0,0,0):1.0,
},
{
    (1,0,0,0,0,0):1.0,
},
{
    (0,0,1,0,0,0):1.0,
}
]

target_coeffs = []

for coeff_dict in target_coeff_dicts:
    coeffs = []
    for degrees in degrees_list:

        if degrees in coeff_dict:
            coeffs.append(coeff_dict[degrees])
        else:
            coeffs.append(0)            
    target_coeffs.append(coeffs)
        
target_coeffs = np.array(target_coeffs, dtype=np.float64)

In [58]:
W = target_coeffs @ np.linalg.pinv(all_coeffs)

In [60]:
W

array([[-1.33005774,  9.35845135, -6.93921182, -3.35022129],
       [-1.12383559,  1.98866049, 12.39526223,  1.84933361],
       [ 0.33435834, -7.77705128, 20.05779229, -6.71732794]])